In [13]:
import re

def extrair_leis(texto):
    terms = re.findall('\s(\d*\.?\\d+\/\\d+)[^\d\/]', texto.lower())
    return list(set(terms))

In [14]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch_dsl import Q, Search

client = Elasticsearch()

search = Search(using=client, index='pa')

result = search.query(Q({"match": {"ementa" : {"query" : "lei"}}}))

response = result.scan()
#response = result.execute()

In [15]:
dict_processo = {
    'numero': [],
    'seq': [],
    'lei': []
}

for h in response:
    leis = extrair_leis(h.ementa)
    for lei in leis:
        # TODO: Normalizar lei: 9.656/98 -> 9.656/1998
        dict_processo['numero'].append(h.numero)
        dict_processo['seq'].append(h.seq)
        dict_processo['lei'].append(lei)


In [16]:
import pandas as pd

df = pd.DataFrame(dict_processo)

df.head()

,numero,seq,lei
0,00159543720068240033,00000,8.078/1990
1,00114985020118240039,00000,12.004/2009
2,40033208820168240000,00000,911/1969
3,03027240620158240010,00000,6.194/1974
4,03027240620158240010,00000,11.482/2007


In [17]:
writer = pd.ExcelWriter('data/leis.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

**Nota:**

Verificar depois a relação com artigos:

CPC
CC
CCom
CPC - Código de Processo Civil
CC - Código Civil
CCom - Código Comercial
ECA - Estatuto da Criança e do Adolescente
LRF - Lei de Recuperação e Falencia
CP - código penal
CPP - Código de proceso penal